In [1]:
import numpy as np 
import pandas as pd
import pickle

In [2]:
lst = {'x1' :[0,1,2,8,9,4,7,8,8,12,11,10,4,5,6,4,5,6,4,5,6,0,1,2,4,5,6] ,'x2':[4,5,6,4,5,7,2,3,8,5,5,6,8,9,10,4,5,6,0,1,2,8,7,3,3,3,7] , 'bi': [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1] ,'y':[1,1,1,1,1,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0]}
df = pd.DataFrame(lst)
df = df.to_numpy()

In [3]:
def check(n,k, mid_w,mid_1_w,out_w):
    y1=np.dot(mid_w , df[k,0:3].reshape(3,1))      
    mid_y = np.zeros((n-u,1))
    for j in range (0,12):
        if(y1[j]>=0):
            mid_y[j]=1
            y1[j]=1
        else:
            mid_y[j]=-1
            y1[j]=-1
    mid_y[12]=1
    y2 = np.dot(mid_1_w , mid_y)
    outs=0
    for j in range (0,5):
        outs += y2[j]*out_w[j]
    outs += out_w[5]*1
    if(outs>=0):
        return 1
    else:
        return -1

In [4]:
def out(c,n,u,i,diff,mid_w,mid_1_w,out_w,output):
    output1=np.dot(mid_w , df[i,0:3].reshape(3,1))         #12*3*3*1==12*1 +1
    mid_out = np.zeros((n-u,1))
    for j in range (0,n-u-1):
        output[j]=output1[j]
        if(output[j]>=0):
            diff[j] = output[j]
            mid_out[j]=1
            output[j]=1
        else:
            diff[j] = -output[j]
            mid_out[j]=-1
            output[j]=-1
    mid_out[n-u-1]=1
    output2=np.dot(mid_1_w , mid_out)              #5*13*13*1 == 5*1
    for j in range (0,u):
        output[12+j]=output2[j]
        if(output[j]>=0):
            diff[12+j] = output[12+j]
            output[12+j]=1
        else:
            diff[12+j] = -output[12+j]
            output[12+j]=-1;
    outs=0
    for j in range (0,u):
        outs += output[12+j]*out_w[j]
    outs += out_w[5]*1
    if(outs>=0):
        output[17]=1
        return 1,diff,mid_w,mid_1_w,out_w,output
    else:
        output[17]=-1
        return -1,diff,mid_w,mid_1_w,out_w,output

In [5]:
def update(c,n,u,i,diff,mid_w,mid_1_w,out_w,output):
    q = np.where(diff==min(diff))
    j = q[0][0]
    inter = (df[i,3]) - (-diff[j])
    if(j<12):
        mid_w[j][0] -= 0.1*inter*df[i,0]
        mid_w[j][1] -= 0.1*inter*df[i,1]
        mid_w[j][2] -= 0.1*inter
    else:
        for l in range(0,n-u-1):
            mid_1_w[j-12][l] -= 0.1*inter*output[l]
        mid_1_w[j-12][12] -= 0.1*inter
    return diff,mid_w,mid_1_w,out_w,output,c

In [6]:
n=18
u=5
flag=0
j=0
c=0
x,y = df.shape
diff = np.zeros(17)
mid_w =np.random.uniform(-1, 1 , (n-u-1,3))
mid_1_w =np.random.uniform(-1, 1 , (u,n-u))
out_w = np.random.uniform(-1, 1 ,n+1)
output = np.zeros(n)
while(flag==0):
    if(c>= 100):
        break;
    i = j%x
    res,diff,mid_w,mid_1_w,out_w,output = out(c,n,u,i,diff ,mid_w,mid_1_w,out_w,output)
    if(res == 1 and df[i,3] ==0):
        diff,mid_w,mid_1_w,out_w,output,c = update(c,n,u,i,diff,mid_w,mid_1_w,out_w,output)
    elif(res == -1 and df[i,3] ==1):
        diff,mid_w,mid_1_w,out_w,output,c = update(c,n,u,i,diff,mid_w,mid_1_w,out_w,output)
    f=0
    for m in range(0,x):
        res1 = check(n,m,mid_w,mid_1_w,out_w)
        if(res1 == 1 and df[m,3] ==0):
            f=1
            break
        elif(res1 == -1 and df[m,3] ==1):
            f=1
            break
            
    if(f==0):
        flag=1;
    c+=1
    j +=1
count=0  



for m in range(0,x):
        res1 = check(n,m, mid_w,mid_1_w,out_w)
        if(res1 == 1 and df[m,3] ==1):
            count +=1
        elif(res1 == -1 and df[m,3] ==0):
            count +=1
print("Accuracy : "+ str(count/x*100))

Accuracy : 51.85185185185185


In [ ]:
model = [n ,mid_w,mid_1_w,out_w]
with open('Madeline.pkl','wb') as f:
    pickle.dump(model, f)

In [8]:
with open('./models/Madeline.pkl','rb') as f:
     model_predict = pickle.load(f)
print("No of neurons: "+str(model_predict[0])+"\n Middle Weights: "+str(model_predict[1])+"\n Output Weights: "+str(model_predict[2]))

No of neurons: 18
 Middle Weights: [[-0.27735364 -0.49877266  0.37275835]
 [-0.43540907 -0.91221376 -0.34231269]
 [ 0.6202432  -0.57327029 -0.47868859]
 [-0.413111   -0.13642695 -0.34809761]
 [-0.00790067 -0.06317465  0.34597849]
 [ 0.39475129 -0.8895448  -0.00414496]
 [-0.08141801  0.15621378  0.68077462]
 [ 0.61998351  0.88569611  0.26204405]
 [ 0.28948045  0.72077551  0.85305208]
 [-0.6763465  -0.34428089  0.30928535]
 [ 0.34065226 -0.96279419  0.39997841]
 [ 0.39246068 -0.08614324  0.95851744]]
 Output Weights: [[-1.18575256e+17 -1.18575256e+17 -3.57834195e+16 -1.18575256e+17
  -2.03723427e+16 -3.70913220e+16  1.18575256e+17  1.18575256e+17
   1.18575256e+17 -1.18575256e+17 -3.70913220e+16  1.18575256e+17
   1.18575256e+17]
 [-8.82568118e-01 -1.85256041e-01 -8.01270489e-01 -5.46965774e-01
   1.79873224e-01  9.10681105e-01 -1.17904211e-01  5.68489922e-02
  -5.96811673e-01 -6.03445771e-01 -7.53641665e-01  1.40988430e-02
  -6.69073869e-01]
 [-8.00543500e-01 -8.51457193e-01 -8.02876427

In [9]:
count=0
for m in range(0,x):
        res1 = check(model_predict[0],m, model_predict[1],model_predict[2],model_predict[3])
        if(res1 == 1 and df[m,3] ==1):
            count +=1
        elif(res1 == -1 and df[m,3] ==0):
            count +=1
print("Accuracy : "+ str(count/x*100))

Accuracy : 55.55555555555556
